# 01-02 : Docker Run

Attempt to run only the function in a docker container. MLRun will still be running only in the local environment.

In [52]:
import os

import mlrun

In [53]:
import logging

# configure log format
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s %(levelname)s [%(name)s] %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

# get the logger
logger = logging.getLogger(__name__)

## 1. Configuration

In [54]:
# set to run locally
MLRUN_LOCAL = False 

# set the artifact path to the local directory
MLRUN_ARTIFACT_PATH = os.path.abspath("../artifacts")

# set the docker image to use
MLRUN_DOCKER_IMAGE = "mlrun/mlrun"

## 1. Set Project

In [55]:
project = mlrun.get_or_create_project(
    name='basic-tests',
    context='./',) 

# set the artifact path to the local directory
project.artifact_path = MLRUN_ARTIFACT_PATH
project.save()

> 2025-05-07 14:33:53,848 [warning] Could not detect path to API server, not connected to API server!
> 2025-05-07 14:33:53,849 [warning] MLRUN_DBPATH is misconfigured. Set this environment variable to the URL of the API server in order to connect
> 2025-05-07 14:33:53,849 [info] Loading project from path: {"path":"./","project_name":"basic-tests","user_project":false}
> 2025-05-07 14:33:53,880 [warning] Could not detect path to API server, not connected to API server!
> 2025-05-07 14:33:53,881 [warning] MLRUN_DBPATH is misconfigured. Set this environment variable to the URL of the API server in order to connect
> 2025-05-07 14:33:53,881 [info] Project loaded successfully: {"path":"./","project_name":"basic-tests","stored_in_db":true}
> 2025-05-07 14:33:53,883 [warning] Could not detect path to API server, not connected to API server!
> 2025-05-07 14:33:53,883 [warning] MLRUN_DBPATH is misconfigured. Set this environment variable to the URL of the API server in order to connect


## 2. Create Function

### 2.1 Create Function Code File

In [56]:
%%writefile hello_world.py

import mlrun

# get the logger
import logging
logger = logging.getLogger(__name__)

def handler(context: mlrun.MLClientCtx = None):
    logger.info("Hello world")

    context.log_result("hello", "world")

Overwriting hello_world.py


### 2.2 Create Project Function

In [57]:
?mlrun.code_to_function

Signature:
mlrun.code_to_function(
    name: Optional[str] = '',
    project: Optional[str] = '',
    tag: Optional[str] = '',
    filename: Optional[str] = '',
    handler: Optional[str] = '',
    kind: Optional[str] = '',
    image: Optional[str] = None,
    code_output: Optional[str] = '',
    embed_code: bool = True,
    description: Optional[str] = '',
    requirements: Union[str, list[str], NoneType] = None,
    categories: Optional[list[str]] = None,
    labels: Optional[dict[str, str]] = None,
    with_doc: Optional[bool] = True,
    ignored_tags: Optional[str] = None,
    requirements_file: Optional[str] = '',
) -> Union[mlrun.runtimes.mpijob.v1.MpiRuntimeV1, mlrun.runtimes.nuclio.function.RemoteRuntime, mlrun.runtimes.nuclio.serving.ServingRuntime, mlrun.runtimes.daskjob.DaskCluster, mlrun.runtimes.kubejob.KubejobRuntime, mlrun.runtimes.local.LocalRuntime, mlrun.runtimes.sparkjob.spark3job.Spark3Runtime, mlrun.runtimes.remotesparkjob.RemoteSparkRuntime, mlrun.runtimes.databri

In [58]:
hello_world = mlrun.code_to_function(
    name='hello-world',
    project='basic-tests',
    filename='hello_world.py',
    handler='handler',
    kind='job',
    image=MLRUN_DOCKER_IMAGE,
    description='Hello world function',
)
hello_world.save()

> 2025-05-07 14:33:53,942 [warning] Could not detect path to API server, not connected to API server!
> 2025-05-07 14:33:53,943 [warning] MLRUN_DBPATH is misconfigured. Set this environment variable to the URL of the API server in order to connect


'db://basic-tests/hello-world'

## 3. Run Function

In [59]:
run = hello_world.run(function=hello_world,
                      handler='handler',
                      local=MLRUN_LOCAL)

> 2025-05-07 14:33:53,953 [warning] Artifact output path is local while no volume mount is specified. Artifacts would not be visible via UI.: {"output_path":"/Users/foulds/code/local-mrlrun/notebooks/artifacts"}
> 2025-05-07 14:33:53,954 [info] Storing function: {"db":"./","name":"hello-world-handler","uid":"b35c58ea350d4bbf8808f39acdb09fa3"}
> 2025-05-07 14:33:53,956 [warning] Could not detect path to API server, not connected to API server!
> 2025-05-07 14:33:53,956 [warning] MLRUN_DBPATH is misconfigured. Set this environment variable to the URL of the API server in order to connect
> 2025-05-07 14:33:53,957 [warning] Could not detect path to API server, not connected to API server!
> 2025-05-07 14:33:53,957 [warning] MLRUN_DBPATH is misconfigured. Set this environment variable to the URL of the API server in order to connect
> 2025-05-07 14:33:53,957 [error] Failed remote run: {"error":"'NopDB' object has no attribute 'submit_job'"}
> 2025-05-07 14:33:53,958 [warning] Could not det

project,uid,iter,start,end,state,kind,name,labels,inputs,parameters,results
basic-tests,...b09fa3,0,,NaT,created,,hello-world-handler,kind=jobowner=foulds,,,


> 2025-05-07 14:33:54,111 [info] Run execution finished: {"name":"hello-world-handler","status":"created"}


In [60]:
# get the run results
logger.info(f"Run ID: {run.metadata.uid}")
logger.info(f"Run state: {run.status.state}")
logger.info(f"Run results: {run.status.results}")

2025-05-07 14:33:54 INFO [__main__] Run ID: b35c58ea350d4bbf8808f39acdb09fa3
2025-05-07 14:33:54 INFO [__main__] Run state: created
2025-05-07 14:33:54 INFO [__main__] Run results: None
